In [2]:
import numpy as np
import pandas as pd

import psutil
import GPUtil

import os,sys,humanize,psutil,GPUtil

import matplotlib.pyplot as plt
import seaborn as sn
from PIL import Image
import matplotlib.image as mpimg
import imageio
import glob
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

%matplotlib inline

import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import ToTensor
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset  # Import the Dataset class

In [4]:
import numpy as np
import wfdb
import glob
import matplotlib.pyplot as plt
from matplotlib.image import imread
import pywt
import pywt.data
import os
from pathlib import Path
from PIL import Image,ImageEnhance
from scipy import fftpack
import shutil
import pandas as pd
import random
import scipy.signal as signal
from wfdb import processing
from itertools import combinations
import ast
import wfdb

In [5]:
def distribute_images():
    test_fold = 10
    
    # 데이터베이스(csv 파일) 로드
    df_db = pd.read_csv('/home/ines/code/ptb-xl/ptbxl_database.csv', index_col='ecg_id')
    # ASMI를 포함하는 인덱스
    asmi_indices = df_db[df_db['scp_codes'].str.contains('ASMI', na=False)].index
    # ASMI가 아닌 인덱스
    normal_indices = df_db[~df_db.index.isin(asmi_indices)].index

    df_st = pd.read_csv('/home/ines/code/ptb-xl/scp_statements.csv', index_col=0)
    df_st = df_st[df_st.diagnostic == 1]


    source_dir = f"/home/ines/code/data_250/data/"
    train_dir_asmi = f'/home/ines/code/data_250/tr_dat/asmi/'
    train_dir_normal = f'/home/ines/code/data_250/tr_dat/normal/'
    test_dir_asmi = f'/home/ines/code/data_250/te_dat/asmi/'
    test_dir_normal = f'/home/ines/code/data_250/te_dat/normal/'
    
    # ASMI 이미지 이동
    for i in asmi_indices:
        five_digit_value = '%05d' % i
        file_name = five_digit_value + "_lr.dat"
        if df_db.loc[i, 'strat_fold'] != test_fold :
            destination_path = os.path.join(train_dir_asmi, file_name)
        else:
            destination_path = os.path.join(test_dir_asmi, file_name)
        shutil.move(os.path.join(source_dir, file_name), destination_path)

    # ASMI가 아닌 이미지 이동
    for n in normal_indices:
        five_digit_value = '%05d' % n
        file_name = five_digit_value + "_lr.dat"
        if df_db.loc[n, 'strat_fold'] != test_fold :
            destination_path = os.path.join(train_dir_normal, file_name)
        else:
            destination_path = os.path.join(test_dir_normal, file_name)
        shutil.move(os.path.join(source_dir, file_name), destination_path)


    # ASMI 이미지 이동
    for i in asmi_indices:
        five_digit_value = '%05d' % i
        file_name = five_digit_value + "_lr.hea"
        if df_db.loc[i, 'strat_fold'] != test_fold :
            destination_path = os.path.join(train_dir_asmi, file_name)
        else:
            destination_path = os.path.join(test_dir_asmi, file_name)
        shutil.move(os.path.join(source_dir, file_name), destination_path)

    # ASMI가 아닌 이미지 이동
    for n in normal_indices:
        five_digit_value = '%05d' % n
        file_name = five_digit_value + "_lr.hea"
        if df_db.loc[n, 'strat_fold'] != test_fold :
            destination_path = os.path.join(train_dir_normal, file_name)
        else:
            destination_path = os.path.join(test_dir_normal, file_name)
        shutil.move(os.path.join(source_dir, file_name), destination_path)

# 이미지 분배 함수 호출
distribute_images()

In [18]:
def downsample_and_save_v1(src_dir, dest_dir, downsample_factor=250):
    """
    src_dir 내의 .dat 파일로부터 V1 채널의 데이터를 추출하여
    downsample_factor에 따라 다운샘플링하고, dest_dir에 .npy 형식으로 저장합니다.

    Args:
    - src_dir (str): 원본 .dat 파일이 위치한 디렉토리 경로.
    - dest_dir (str): 저장할 .npy 파일이 위치할 디렉토리 경로.
    - downsample_factor (int): 다운샘플링할 샘플의 수.
    """
    # .hea 파일에서 V1 채널의 인덱스를 찾습니다.
    hea_path = glob.glob(os.path.join(src_dir, '*.hea'))[0]
    with open(hea_path, 'r') as hea_file:
        lines = hea_file.readlines()
    
    v1_index = None
    for i, line in enumerate(lines):
        if 'V1' in line:
            v1_index = i - 1  # .hea 파일은 1부터 시작하는 인덱스를 사용하지만 Python은 0부터 시작합니다.
            break
    
    if v1_index is None:
        raise ValueError("V1 채널을 찾을 수 없습니다.")
    
    # .dat 파일을 로드하고 V1 채널만 추출합니다.
    dat_files = glob.glob(os.path.join(src_dir, '*.dat'))
    for file_path in dat_files:
        filename = os.path.basename(file_path).replace('.dat', '')
        data = np.fromfile(file_path, dtype=np.int16)  # 데이터 타입이 int16이라고 가정합니다.
        channel_data = data[v1_index::12]  # 채널 수(12)만큼 스텝을 건너뛰면서 V1 데이터만 추출합니다.
        
        # 다운샘플링
        downsampled_data = resample(channel_data, downsample_factor)
        
        # .npy로 저장
        np.save(os.path.join(dest_dir, f'{filename}_V1.npy'), downsampled_data)

# 소스 디렉토리와 목적지 디렉토리 경로 설정
src_dir = '/home/ines/code/data_250/te_dat/asmi/'  # 예: '/path/to/your/source_directory'
dest_dir = '/home/ines/code/data_250/V1_250_te/asmi/'  # 예: '/path/to/your/destination_directory'

downsample_and_save_v1(src_dir, dest_dir)